In [1]:
%pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install ipython-sql
%pip install ipython-sql prettytable

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/4 [ipython-genutils]
   ---------------------------------------- 0/4 [ipython-genutils]
   ---------------------------------------- 0/4 [ipython-genutils]
   ---------- ----------------------------- 1/4 [sqlparse]
   ---------- ----------------------------- 1/4 [sqlparse]
   ---------- ----------------------------- 1/4 [sqlparse]
   ---------- ----------------------------- 1/4 [sqlparse]
   ---------- ----------------------------- 1/4 [sqlparse]
   ---------- ----------------------------- 1/4 [sqlparse]
   ---------- ----------------------------- 1/4 [sqlparse]
   -------------------- ------------------- 2/4 [prettytable]
   ------------------------------ --------- 3/4 [ipython-sql]
   ---------------------------------------- 4/4 [ipython-sql]

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv, sqlite3
import prettytable
import pandas as pd

### Connect to the database

Let us first load the SQL extension and establish a connection with the database

In [3]:
%reload_ext sql

In [4]:
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
%sql sqlite:///my_data1.db

In [6]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [7]:
#DROP THE BLANK TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [8]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

### Task 1




##### Display the names of the unique launch sites  in the space mission

In [9]:
%sql SELECT DISTINCT Launch_Site FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA

In [10]:
%sql SELECT Launch_Site FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [11]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) AS total_payload FROM SPACEXTBL WHERE Customer LIKE 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


total_payload
45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1

In [12]:
%sql SELECT avg(PAYLOAD_MASS__KG_) AS Avg_Payload FROM SPACEXTBL WHERE Booster_Version LIKE 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Avg_Payload
2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 

In [13]:
%sql SELECT min(date) AS Early_Date from SPACEXTBL where Landing_Outcome LIKE 'Success (ground pad)'

 * sqlite:///my_data1.db
Done.


Early_Date
2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [14]:
%sql SELECT DISTINCT Customer, Landing_Outcome,PAYLOAD_MASS__KG_ FROM SPACEXTBL WHERE Landing_Outcome ='Success (drone ship)' AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Customer,Landing_Outcome,PAYLOAD_MASS__KG_
SKY Perfect JSAT Group,Success (drone ship),4696
SKY Perfect JSAT Group,Success (drone ship),4600
SES,Success (drone ship),5300
SES EchoStar,Success (drone ship),5200


### Task 7

##### List the total number of successful and failure mission outcomes

In [15]:
%sql SELECT Mission_Outcome, Count(*) AS Numbers FROM SPACEXTBL GROUP BY Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Numbers
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### Task 8

##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.

In [16]:
%sql SELECT Booster_Version, Max_Payload FROM (SELECT Booster_Version, MAX(PAYLOAD_MASS__KG_) AS Max_Payload FROM SPACEXTBL GROUP BY Booster_Version) AS Sub;

 * sqlite:///my_data1.db
Done.


Booster_Version,Max_Payload
F9 B4 B1039.2,2647
F9 B4 B1040.2,5384
F9 B4 B1041.2,9600
F9 B4 B1043.2,6460
F9 B4 B1039.1,3310
F9 B4 B1040.1,4990
F9 B4 B1041.1,9600
F9 B4 B1042.1,3500
F9 B4 B1043.1,5000
F9 B4 B1044,6092


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**

In [24]:
%sql SELECT SUBSTR(Date, 6, 2) AS Month, Booster_Version, Launch_Site, Landing_Outcome FROM SPACEXTBL WHERE Landing_Outcome LIKE 'Failure%drone%' AND SUBSTR(Date, 0, 5) = '2015';

 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Launch_Site,Landing_Outcome
01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


### Task 10


##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [19]:
%sql SELECT Landing_Outcome, COUNT(*) AS Numbers FROM SPACEXTBL WHERE Landing_Outcome LIKE 'Success%' AND Date BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY Landing_Outcome ORDER BY Numbers DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Numbers
Success (drone ship),5
Success (ground pad),3
